In [4]:
import torch

def similarity_transform(from_points, to_points):
    
   
    N, m = from_points.shape
    
    mean_from = (from_points.mean(dim=0)).double()
    mean_to = (to_points.mean(dim=0)).double()
    
    delta_from = (from_points - mean_from).double() # N x m
    delta_to = (to_points - mean_to).double()       # N x m
    
    sigma_from = (delta_from * delta_from).sum(dim=1).mean()
    sigma_to = (delta_to * delta_to).sum(dim=1).mean()
    
    cov_matrix = delta_to.T @ delta_from / N
    
    U, d, V_t = torch.linalg.svd(cov_matrix, full_matrices = True)
    cov_rank = torch.linalg.matrix_rank(cov_matrix)
    S = torch.eye(m,dtype=torch.float64)

    
    if cov_rank >= m - 1 and torch.det(cov_matrix) < 0:
        S[m-1, m-1] = -1
    elif cov_rank < m-1:
        raise ValueError("colinearility detected in covariance matrix:\n{}".format(cov_matrix))
    
    R = U @ S @ V_t
    c = (d * S.diagonal()).sum() / sigma_from
    t = mean_to - c*R @ mean_from
    
    return c*R, t

if __name__ == "__main__":
    ## Testing case from the original paper by Umeyama.
    # each row represents a point
    from_points = torch.tensor([[0, 0],
                                [1, 0],
                                [0, 2]], dtype=torch.float64)
    to_points = torch.tensor([[0, 0],
                              [-1, 0],
                              [0, 2]], dtype=torch.float64)
    
    c_ans = 0.721
    R_ans = torch.tensor([[0.832,  0.555],
                          [-0.555, 0.832]], dtype=torch.float64)
    t_ans = torch.tensor([-0.8, 0.4], dtype=torch.float64)
    M_ans = c_ans*R_ans
    
    M, t = similarity_transform(from_points, to_points)
  
    assert torch.allclose(M, M_ans, atol = 1e-3) and torch.allclose(t, t_ans, atol = 1e-3), "test from Umeyama's paper fail"
    print("test from Umeyama's paper pass")
    
    # test in 3D space
    from_points = torch.tensor([[0, 0, 1],
                                [1, 0, 3],
                                [2, 5, 8]], dtype=torch.float64)

    M_ans = torch.tensor([[0, -1, 0],
                          [1,  0, 0],
                          [0,  0, 1]], dtype=torch.float64)
    t_ans = torch.tensor([1, 1, 1], dtype=torch.float64)

    to_points = M_ans.matmul(from_points.T).T + t_ans
    
    
    M, t = similarity_transform(from_points, to_points)
    assert torch.allclose(M, M_ans) and torch.allclose(t, t_ans), "test for 3D space fail"
    print("test for 3D space pass")

test from Umeyama's paper pass
test for 3D space pass
